## Importing all the libraries

In [1]:
#Importing the libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
final = pd.read_csv("16thFeb2022_100k_120k_data_merged_with_sf_v2.csv")

In [3]:
final.drop(columns =['Unnamed: 0','code','l1_finish_time','l1_pc','DXCAT3',
                    'query_source','telehealth_encounter',
                    'DX Category','DX Context'],axis = 1,inplace = True)

In [4]:
final.shape

(2088529, 40)

In [5]:
for column in final:
    if final[column].dtype == 'float64':
        final[column] = pd.to_numeric(final[column], downcast = 'float')
    if final[column].dtype == 'int64':
        final[column] = pd.to_numeric(final[column], downcast = 'unsigned')

In [6]:
final.columns

Index(['code1', 'final_codes', 'DX Codetype', 'DX Condition Type', 'enc_id',
       'meat', 'text', 'page', 'section', 'NLP_VERSION',
       'parent_rxcui_mapping', 'final_code_family', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count', 'project_id', 'output'],
      dtype='object')

In [7]:
final[final['dos_start'].isna()].shape

(209, 40)

In [8]:
final.dropna(axis=0, subset=['dos_start','dos_end'],inplace = True)

In [9]:
for column in final:
    if final[column].dtype == 'float64':
        final[column] = pd.to_numeric(final[column], downcast = 'float')
    if final[column].dtype == 'int64':
        final[column] = pd.to_numeric(final[column], downcast = 'unsigned')

## Creating New Features

#### Features related to meat

In [10]:
final.meat.value_counts().head()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [11]:
final["Has MEAT"] = np.where(final["meat"] != "[]",1,0)

In [12]:
final["Has MEAT"].value_counts()

1    1572738
0     515582
Name: Has MEAT, dtype: int64

In [13]:
final.columns

Index(['code1', 'final_codes', 'DX Codetype', 'DX Condition Type', 'enc_id',
       'meat', 'text', 'page', 'section', 'NLP_VERSION',
       'parent_rxcui_mapping', 'final_code_family', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count', 'project_id', 'output', 'Has MEAT'],
      dtype='object')

In [14]:
final["rx_count"].value_counts()

0      947306
1      151846
2      149066
3      120085
4      112248
        ...  
128         1
260         1
265         1
208         1
194         1
Name: rx_count, Length: 192, dtype: int64

In [15]:
final["Has RX MEAT"] = np.where((final["rx_count"] != 0) | (final["medicine_count"] != 0),1,0)

In [16]:
final["Has RX MEAT"].value_counts()

1    1366083
0     722237
Name: Has RX MEAT, dtype: int64

In [17]:
final["Has Procedure MEAT"] = np.where((final["procedure_count"] != 0) | (final["procedures_count"] != 0) ,1,0)

In [18]:
final["Has Labs MEAT"] = np.where((final["labs_count"] != 0) | (final["labtest_count"] != 0),1,0)

In [19]:
final["Has RX MEAT"].value_counts()

1    1366083
0     722237
Name: Has RX MEAT, dtype: int64

In [20]:
final["Has Labs MEAT"].value_counts()

0    1359708
1     728612
Name: Has Labs MEAT, dtype: int64

#### Other Features

In [21]:
number_of_encounters_per_chart = pd.DataFrame(final.groupby("chart_id").enc_id.nunique()).reset_index()

In [22]:
number_of_encounters_per_chart.columns = ["chart_id","number_of_encounters_per_chart"]

In [23]:
number_of_encounters_per_chart

,chart_id,number_of_encounters_per_chart
0,39358,21
1,39388,5
2,39400,2
3,393645,5
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,1
19944,5382,3
19945,5387858,1
19946,539049,10


In [24]:
final = final.merge(number_of_encounters_per_chart, on = 'chart_id', how='left')

In [25]:
number_of_codes_per_chart = pd.DataFrame(final.groupby("chart_id").code1.nunique()).reset_index()

In [26]:
number_of_codes_per_chart.columns = ["chart_id","number_of_code1_per_chart"]

In [27]:
number_of_codes_per_chart

,chart_id,number_of_code1_per_chart
0,39358,2
1,39388,7
2,39400,3
3,393645,14
4,393649,11
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,6
19945,5387858,1
19946,539049,6


In [28]:
final = final.merge(number_of_codes_per_chart, on = 'chart_id', how='left')

In [29]:
number_of_occurences_of_code1_per_chart = pd.DataFrame(final.reset_index().groupby(["chart_id","code1"]).index.count()).reset_index()

In [30]:
number_of_occurences_of_code1_per_chart.head(10)

,chart_id,code1,index
0,39358,G809,186
1,39358,Z684,19
2,39388,I10,28
3,39388,I5030,3
4,39388,I509,56
5,39388,L98499,5
6,39388,N189,13
7,39388,R569,5
8,39388,Z684,5
9,39400,E119,2


In [31]:
number_of_occurences_of_code1_per_chart.columns = ["chart_id","code1","number_of_occurences_of_code1_per_chart"]

In [32]:
number_of_occurences_of_code1_per_chart.head(10)

,chart_id,code1,number_of_occurences_of_code1_per_chart
0,39358,G809,186
1,39358,Z684,19
2,39388,I10,28
3,39388,I5030,3
4,39388,I509,56
5,39388,L98499,5
6,39388,N189,13
7,39388,R569,5
8,39388,Z684,5
9,39400,E119,2


In [33]:
final = final.merge(number_of_occurences_of_code1_per_chart, on = ['chart_id','code1'], how='left')

In [34]:
final["DX incidence per encounter"] = final["number_of_occurences_of_code1_per_chart"]/final["number_of_encounters_per_chart"]

In [35]:
final["DX incidence per encounter"]

0          4.000000
1          4.000000
2          1.000000
3          4.000000
4          0.333333
             ...   
2088315    4.000000
2088316    4.000000
2088317    3.000000
2088318    4.000000
2088319    4.000000
Name: DX incidence per encounter, Length: 2088320, dtype: float64

In [36]:
number_of_encounters_per_chart_and_code1 = pd.DataFrame(final.groupby(["chart_id","code1"]).enc_id.nunique()).reset_index()

In [37]:
number_of_encounters_per_chart_and_code1.columns = ["chart_id","code1","number_of_encounters_per_chart_and_code1"]

In [38]:
number_of_encounters_per_chart_and_code1

,chart_id,code1,number_of_encounters_per_chart_and_code1
0,39358,G809,21
1,39358,Z684,19
2,39388,I10,5
3,39388,I5030,3
4,39388,I509,5
...,...,...,...
183776,5390815-1LR-2021-22,E162,1
183777,5390815-1LR-2021-22,I213,1
183778,5390815-1LR-2021-22,I214,1
183779,5390815-1LR-2021-22,I219,1


In [39]:
final = final.merge(number_of_encounters_per_chart_and_code1, on = ['chart_id','code1'], how='left')

In [40]:
number_of_pages_per_chart_per_enc = pd.DataFrame(final.groupby(["chart_id","enc_id"]).page.nunique()).reset_index()

In [41]:
number_of_pages_per_chart_per_enc.columns = ["chart_id","enc_id","number_of_pages_per_chart_per_enc"]

In [42]:
number_of_pages_per_chart_per_enc

,chart_id,enc_id,number_of_pages_per_chart_per_enc
0,39358,1,4
1,39358,2,3
2,39358,3,4
3,39358,4,3
4,39358,5,3
...,...,...,...
162909,5390815-1LR-2021-22,0,4
162910,5390815-1LR-2021-22,1,4
162911,5390815-1LR-2021-22,2,2
162912,5390815-1LR-2021-22,3,2


In [43]:
final = final.merge(number_of_pages_per_chart_per_enc, on = ['chart_id','enc_id'], how='left')

In [44]:
number_of_pages_in_a_chart = pd.DataFrame(final.groupby("chart_id").page.nunique()).reset_index()

In [45]:
number_of_pages_in_a_chart.columns = ["chart_id","number_of_pages_in_a_chart"]

In [46]:
number_of_pages_in_a_chart

,chart_id,number_of_pages_in_a_chart
0,39358,60
1,39388,19
2,39400,7
3,393645,26
4,393649,14
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,8
19945,5387858,2
19946,539049,18


In [47]:
final = final.merge(number_of_pages_in_a_chart, on = 'chart_id', how='left')

In [48]:
number_of_unique_sections_in_a_chart = pd.DataFrame(final.groupby("chart_id").section.nunique()).reset_index()

In [49]:
number_of_unique_sections_in_a_chart.columns = ["chart_id","number_of_unique_sections_in_a_chart"]

In [50]:
number_of_unique_sections_in_a_chart

,chart_id,number_of_unique_sections_in_a_chart
0,39358,8
1,39388,9
2,39400,5
3,393645,12
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,4
19944,5382,7
19945,5387858,2
19946,539049,5


In [51]:
final = final.merge(number_of_unique_sections_in_a_chart, on = 'chart_id', how='left')

In [52]:
import ast

In [53]:
final["meat"] = final["meat"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [54]:
final["number_of_meat_per_Dx"] = final["meat"].apply(lambda x: len(x))

In [55]:
final["final_codes"] = final["final_codes"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [56]:
final["number_of_unique_codes_in_final_codes_per_Dx"] = final["final_codes"].apply(lambda x: len(set(x)))

In [57]:
final["parent_rxcui_mapping"] = final["parent_rxcui_mapping"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [58]:
final["parent_rxcui_mapping"][0]

[6809]

In [59]:
final["number_of_unique_parent_rxcui_mapping_per_Dx"] = final["parent_rxcui_mapping"].apply(lambda x: len(set(x)))

In [60]:
final["code1_family"] = final["code1"].apply(lambda x: x[:3] if x else None)

In [61]:
number_of_code1_family_per_chart = pd.DataFrame(final.groupby("chart_id").code1_family.nunique()).reset_index()

In [62]:
number_of_code1_family_per_chart.columns = ["chart_id","number_of_code1_family_per_chart"]

In [63]:
number_of_code1_family_per_chart

,chart_id,number_of_code1_family_per_chart
0,39358,2
1,39388,6
2,39400,3
3,393645,11
4,393649,7
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,6
19945,5387858,1
19946,539049,6


In [64]:
final = final.merge(number_of_code1_family_per_chart, on = 'chart_id', how='left')

In [65]:
final["Acute Condition"] = np.where(final['DX Condition Type'] == "Acute",1,0)

In [66]:
number_of_acute_conditions_per_chart = pd.DataFrame(final.groupby("chart_id")["Acute Condition"].sum()).reset_index()

In [67]:
number_of_acute_conditions_per_chart.columns = ["chart_id","number_of_acute_conditions_per_chart"]

In [68]:
number_of_acute_conditions_per_chart

,chart_id,number_of_acute_conditions_per_chart
0,39358,0
1,39388,5
2,39400,12
3,393645,22
4,393649,66
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,0
19944,5382,8
19945,5387858,0
19946,539049,3


In [69]:
final = final.merge(number_of_acute_conditions_per_chart, on = 'chart_id', how='left')

In [70]:
final["Chronic Condition"] = np.where(final['DX Condition Type'] == "Chronic",1,0)

In [71]:
number_of_chronic_conditions_per_chart = pd.DataFrame(final.groupby("chart_id")["Chronic Condition"].sum()).reset_index()

In [72]:
number_of_chronic_conditions_per_chart.columns = ["chart_id","number_of_chronic_conditions_per_chart"]

In [73]:
number_of_chronic_conditions_per_chart

,chart_id,number_of_chronic_conditions_per_chart
0,39358,63
1,39388,97
2,39400,4
3,393645,162
4,393649,125
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,4
19944,5382,10
19945,5387858,4
19946,539049,18


In [74]:
final = final.merge(number_of_chronic_conditions_per_chart, on = 'chart_id', how='left')

#### Features realted to DOS created from dos_start

In [75]:
final['dos_start_year'] = pd.DatetimeIndex(final['dos_start']).year

In [76]:
final['dos_start'] = pd.to_datetime(final['dos_start'])

In [77]:
final['dos_start_quarter'] = pd.PeriodIndex(final['dos_start'], freq='Q')

In [78]:
number_of_unique_dos_per_chart = pd.DataFrame(final.groupby("chart_id").dos_start.nunique()).reset_index()

In [79]:
number_of_unique_dos_per_chart.columns = ["chart_id","number_of_unique_dos_start_per_chart"]

In [80]:
number_of_unique_dos_per_chart

,chart_id,number_of_unique_dos_start_per_chart
0,39358,21
1,39388,5
2,39400,1
3,393645,5
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,1
19944,5382,3
19945,5387858,1
19946,539049,10


In [81]:
final = final.merge(number_of_unique_dos_per_chart, on = 'chart_id', how='left')

#### Dx occured in different years

In [82]:
dos_in_different_year = pd.DataFrame(final.groupby(['chart_id','code1']).dos_start_year.nunique().reset_index())

In [83]:
dos_in_different_year.columns = ['chart_id','code1','dos_count_of_different_year_per_chart_and_code1']

In [84]:
final = final.merge(dos_in_different_year, on = ['chart_id','code1'], how='left')

In [85]:
final["dos_in_different_year_per_chart_per_dx"] = np.where(final["dos_count_of_different_year_per_chart_and_code1"] >1,1,0)

#### Dx occured in different quarters

In [86]:
dos_in_different_quarter = pd.DataFrame(final.groupby(['chart_id','code1']).dos_start_quarter.nunique().reset_index())

In [87]:
dos_in_different_quarter.columns = ['chart_id','code1','dos_count_of_different_quarter_per_chart_and_code1']

In [88]:
final = final.merge(dos_in_different_quarter, on = ['chart_id','code1'], how='left')

In [89]:
final["dos_in_different_quarter_per_chart_per_dx"] = np.where(final["dos_count_of_different_quarter_per_chart_and_code1"] >1,1,0)

In [90]:
percentage_of_autocodable_codes_per_chart = pd.DataFrame(round(final.groupby("chart_id").output.sum()/final.groupby("chart_id").output.count(),2).reset_index())

In [91]:
percentage_of_autocodable_codes_per_chart.columns = ['chart_id','percentage_of_autocodable_codes_per_chart']

In [92]:
percentage_of_autocodable_codes_per_chart

,chart_id,percentage_of_autocodable_codes_per_chart
0,39358,0.91
1,39388,0.49
2,39400,0.75
3,393645,0.80
4,393649,0.72
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,0.50
19944,5382,0.44
19945,5387858,1.00
19946,539049,0.78


In [93]:
final = final.merge(percentage_of_autocodable_codes_per_chart, on = 'chart_id', how='left')

In [104]:
#final.to_csv("19thFeb2022_100k_120k_data_merged_with_sf_and_new_features_v2.csv")

In [1]:
import os

In [5]:
os.remove("CIOX_RAHIX_IC20190521142219_591M7914320_124850035.json")

In [ ]:
a = [   "CIOX_RAHIX_IC20190521142219_012M6291820_124836556.json",
        CIOX_RAHIX_IC20190521142219_014M8322040_124838067.json",
        CIOX_RAHIX_IC20190521142219_015M7855410_124843568.json",
        CIOX_RAHIX_IC20190521142219_024A7967120_124850658.json",
        CIOX_RAHIX_IC20190521142219_024M8275510_124837244.json",
        CIOX_RAHIX_IC20190521142219_026M9276310_124838029.json",
        CIOX_RAHIX_IC20190521142219_028M7971420_124847182.json",
        CIOX_RAHIX_IC20190521142219_029A6375620_124852610.json",
        CIOX_RAHIX_IC20190521142219_029M8826530_124850860.json",
        CIOX_RAHIX_IC20190521142219_032M8903310_124852103.json",
        CIOX_RAHIX_IC20190521142219_052M8236420_124855093.json",
        CIOX_RAHIX_IC20190521142219_056M9946920_124853006.json",
        CIOX_RAHIX_IC20190521142219_059M8770530_124836955.json",
        CIOX_RAHIX_IC20190521142219_059M8863510_124854378.json",
        CIOX_RAHIX_IC20190521142219_059M8916020_124836759.json",
        CIOX_RAHIX_IC20190521142219_061A7391510_124859874.json",
        CIOX_RAHIX_IC20190521142219_065M9483320_124838474.json",
        CIOX_RAHIX_IC20190521142219_075M5799410_124850118.json
        CIOX_RAHIX_IC20190521142219_080A8015630_124859098.json
        CIOX_RAHIX_IC20190521142219_080M8814020_124837010.json
        CIOX_RAHIX_IC20190521142219_082M7600740_124846262.json
        CIOX_RAHIX_IC20190521142219_083M7196450_124845040.json
        CIOX_RAHIX_IC20190521142219_084M9268620_124836493.json
        CIOX_RAHIX_IC20190521142219_086A5143820_124841415.json
        CIOX_RAHIX_IC20190521142219_094M9194610_124838376.json
        CIOX_RAHIX_IC20190521142219_095M5815720_124853635.json
        CIOX_RAHIX_IC20190521142219_098M5816110_124849950.json
        CIOX_RAHIX_IC20190521142219_098M8838640_124850551.json
        CIOX_RAHIX_IC20190521142219_105M5353910_124838166.json
        CIOX_RAHIX_IC20190521142219_107M8807740_124836335.json
        CIOX_RAHIX_IC20190521142219_110M5848320_124859664.json
        CIOX_RAHIX_IC20190521142219_120M5797020_124856271.json
        CIOX_RAHIX_IC20190521142219_120M8814220_124847461.json
        CIOX_RAHIX_IC20190521142219_121A6816450_124858069.json
        CIOX_RAHIX_IC20190521142219_130M7313310_124854120.json
        CIOX_RAHIX_IC20190521142219_148A7996650_124840241.json
        CIOX_RAHIX_IC20190521142219_166M5848720_124840645.json
        CIOX_RAHIX_IC20190521142219_168M5355310_124856772.json
        CIOX_RAHIX_IC20190521142219_173A6256910_124848639.json
        CIOX_RAHIX_IC20190521142219_176M8806210_124837346.json
        CIOX_RAHIX_IC20190521142219_181A5196310_124856743.json
        CIOX_RAHIX_IC20190521142219_194M7834310_124846824.json
        CIOX_RAHIX_IC20190521142219_202M7604520_124854885.json
        CIOX_RAHIX_IC20190521142219_206M7344430_124840117.json
        CIOX_RAHIX_IC20190521142219_209A7538010_124848292.json
        CIOX_RAHIX_IC20190521142219_213A8020210_124836598.json
        CIOX_RAHIX_IC20190521142219_215M6057920_124840135.json
        CIOX_RAHIX_IC20190521142219_230M9094620_124845046.json
        CIOX_RAHIX_IC20190521142219_234M5357010_124842497.json
        CIOX_RAHIX_IC20190521142219_254M7905510_124838726.json
        CIOX_RAHIX_IC20190521142219_254M8775010_124838978.json
        CIOX_RAHIX_IC20190521142219_255M8817140_124837856.json
        CIOX_RAHIX_IC20190521142219_257A5452120_124849934.json
        CIOX_RAHIX_IC20190521142219_267M6935320_124850738.json
        CIOX_RAHIX_IC20190521142219_268M5686120_124850737.json
        CIOX_RAHIX_IC20190521142219_277M8340330_124852792.json
        CIOX_RAHIX_IC20190521142219_281M5748010_124860004.json
        CIOX_RAHIX_IC20190521142219_291M8012910_124858109.json
        CIOX_RAHIX_IC20190521142219_297M5827320_124842157.json
        CIOX_RAHIX_IC20190521142219_297M5852910_124845845.json
        CIOX_RAHIX_IC20190521142219_299M8816120_124857776.json
        CIOX_RAHIX_IC20190521142219_299M9208910_124846056.json
        CIOX_RAHIX_IC20190521142219_301A7990010_124839901.json
        CIOX_RAHIX_IC20190521142219_306M9305020_124846264.json
        CIOX_RAHIX_IC20190521142219_311M9234840_124844959.json
        CIOX_RAHIX_IC20190521142219_316A8026520_124846755.json
        CIOX_RAHIX_IC20190521142219_329A7156610_124844482.json
        CIOX_RAHIX_IC20190521142219_329M8868510_124859484.json
        CIOX_RAHIX_IC20190521142219_333M5686520_124843637.json
        CIOX_RAHIX_IC20190521142219_334M7904810_124848070.json
        CIOX_RAHIX_IC20190521142219_337M5690410_124836188.json
        CIOX_RAHIX_IC20190521142219_348A6497140_124836744.json
        CIOX_RAHIX_IC20190521142219_348A7969420_124839923.json
        CIOX_RAHIX_IC20190521142219_349A7118620_124857441.json
        CIOX_RAHIX_IC20190521142219_350A6286540_124839331.json
        CIOX_RAHIX_IC20190521142219_360M5770110_124856957.json
        CIOX_RAHIX_IC20190521142219_361M6611210_124846489.json
        CIOX_RAHIX_IC20190521142219_368M9342020_124857474.json
        CIOX_RAHIX_IC20190521142219_371M5858320_124843418.json
        CIOX_RAHIX_IC20190521142219_380M5708720_124839697.json
        CIOX_RAHIX_IC20190521142219_380M8232410_124850842.json
        CIOX_RAHIX_IC20190521142219_390M7972620_124852767.json
        CIOX_RAHIX_IC20190521142219_394M8737110_124850820.json
        CIOX_RAHIX_IC20190521142219_399A8006920_124853354.json
        CIOX_RAHIX_IC20190521142219_402M5776810_124844600.json
        CIOX_RAHIX_IC20190521142219_404M5518620_124838611.json
        CIOX_RAHIX_IC20190521142219_406M5740430_124839179.json
        CIOX_RAHIX_IC20190521142219_424A7005320_124857470.json
        CIOX_RAHIX_IC20190521142219_425M5777320_124845087.json
        CIOX_RAHIX_IC20190521142219_426M9462120_124846370.json
        CIOX_RAHIX_IC20190521142219_428A7573510_124837024.json
        CIOX_RAHIX_IC20190521142219_433M5359540_124838220.json
        CIOX_RAHIX_IC20190521142219_434M8607840_124846187.json
        CIOX_RAHIX_IC20190521142219_442M5355230_124854664.json
        CIOX_RAHIX_IC20190521142219_447A8010320_124853153.json
        CIOX_RAHIX_IC20190521142219_462M5499410_124857882.json
        CIOX_RAHIX_IC20190521142219_464M5969940_124844455.json
        CIOX_RAHIX_IC20190521142219_464M6239820_124847038.json
        CIOX_RAHIX_IC20190521142219_466M5849940_124840678.json
        CIOX_RAHIX_IC20190521142219_467A7589140_124836530.json
        CIOX_RAHIX_IC20190521142219_467M6225350_124850629.json
        CIOX_RAHIX_IC20190521142219_470M9296470_124844314.json
        CIOX_RAHIX_IC20190521142219_475A6238610_124842428.json
        CIOX_RAHIX_IC20190521142219_477M8326550_124852592.json
        CIOX_RAHIX_IC20190521142219_479M5812050_124849280.json
        CIOX_RAHIX_IC20190521142219_479M9288710_124838462.json
        CIOX_RAHIX_IC20190521142219_491A5064620_124842403.json
        CIOX_RAHIX_IC20190521142219_502M9900220_124845253.json
        CIOX_RAHIX_IC20190521142219_503M8891420_124846129.json
        CIOX_RAHIX_IC20190521142219_506M8192320_124836126.json
        CIOX_RAHIX_IC20190521142219_509M5826110_124855458.json
        CIOX_RAHIX_IC20190521142219_512M7794710_124848988.json
        CIOX_RAHIX_IC20190521142219_520M9109410_124849563.json
        CIOX_RAHIX_IC20190521142219_529M9867710_124837406.json
        CIOX_RAHIX_IC20190521142219_534M7900720_124859742.json
        CIOX_RAHIX_IC20190521142219_535M9584120_124840000.json
        CIOX_RAHIX_IC20190521142219_536A7975640_124855275.json
        CIOX_RAHIX_IC20190521142219_540M6712320_124854988.json
        CIOX_RAHIX_IC20190521142219_544A7608930_124853060.json
        CIOX_RAHIX_IC20190521142219_549M6189020_124839464.json
        CIOX_RAHIX_IC20190521142219_556A7902520_124859870.json
        CIOX_RAHIX_IC20190521142219_563A7586010_124845031.json
        CIOX_RAHIX_IC20190521142219_576M7898120_124859087.json
        CIOX_RAHIX_IC20190521142219_580A6401670_124857962.json
        CIOX_RAHIX_IC20190521142219_593M7354410_124856219.json
        CIOX_RAHIX_IC20190521142219_600M5842110_124846917.json
        CIOX_RAHIX_IC20190521142219_605A6998130_124850055.json
        CIOX_RAHIX_IC20190521142219_607A6342340_124855490.json
        CIOX_RAHIX_IC20190521142219_607M7759520_124841320.json
        CIOX_RAHIX_IC20190521142219_608M7267220_124844484.json
        CIOX_RAHIX_IC20190521142219_611M5499220_124844258.json
        CIOX_RAHIX_IC20190521142219_614A7033651_124841865.json
        CIOX_RAHIX_IC20190521142219_616M6450840_124836852.json
        CIOX_RAHIX_IC20190521142219_619M5348420_124837523.json
        CIOX_RAHIX_IC20190521142219_619M9787120_124841979.json
        CIOX_RAHIX_IC20190521142219_623A6454320_124841156.json
        CIOX_RAHIX_IC20190521142219_637M5687210_124843020.json
        CIOX_RAHIX_IC20190521142219_646M8771710_124857779.json
        CIOX_RAHIX_IC20190521142219_649M8608940_124851011.json
        CIOX_RAHIX_IC20190521142219_653M5773410_124849417.json
        CIOX_RAHIX_IC20190521142219_654A5156310_124853860.json
        CIOX_RAHIX_IC20190521142219_654M8765420_124858743.json
        CIOX_RAHIX_IC20190521142219_656M7755110_124859483.json
        CIOX_RAHIX_IC20190521142219_657A5831910_124858925.json
        CIOX_RAHIX_IC20190521142219_661M5360310_124853694.json
        CIOX_RAHIX_IC20190521142219_661M9907220_124853141.json
        CIOX_RAHIX_IC20190521142219_663A6358910_124836273.json
        CIOX_RAHIX_IC20190521142219_663M5343350_124856948.json
        CIOX_RAHIX_IC20190521142219_664A5912720_124847988.json
        CIOX_RAHIX_IC20190521142219_665A8005940_124845369.json
        CIOX_RAHIX_IC20190521142219_674M6413410_124846703.json
        CIOX_RAHIX_IC20190521142219_677A6488220_124854313.json
        CIOX_RAHIX_IC20190521142219_683M8759810_124846898.json
        CIOX_RAHIX_IC20190521142219_686A7975620_124844183.json
        CIOX_RAHIX_IC20190521142219_697M5843740_124842267.json
        CIOX_RAHIX_IC20190521142219_697M9827020_124855114.json
        CIOX_RAHIX_IC20190521142219_705M9249610_124847082.json
        CIOX_RAHIX_IC20190521142219_708A8008841_124855552.json
        CIOX_RAHIX_IC20190521142219_709A7830320_124850969.json
        CIOX_RAHIX_IC20190521142219_709A7877910_124844418.json
        CIOX_RAHIX_IC20190521142219_713M5101820_124852807.json
        CIOX_RAHIX_IC20190521142219_714M6537210_124840253.json
        CIOX_RAHIX_IC20190521142219_714M7919720_124836266.json
        CIOX_RAHIX_IC20190521142219_715A7657010_124845585.json
        CIOX_RAHIX_IC20190521142219_715A7998330_124840792.json
        CIOX_RAHIX_IC20190521142219_718A5388310_124856627.json
        CIOX_RAHIX_IC20190521142219_722M6665030_124848591.json
        CIOX_RAHIX_IC20190521142219_723A6314520_124843383.json
        CIOX_RAHIX_IC20190521142219_727M6123220_124849902.json
        CIOX_RAHIX_IC20190521142219_731M7929620_124859114.json
        CIOX_RAHIX_IC20190521142219_735A7689450_124849443.json
        CIOX_RAHIX_IC20190521142219_736A5188910_124836850.json
        CIOX_RAHIX_IC20190521142219_736M9186920_124838198.json
        CIOX_RAHIX_IC20190521142219_739M7801820_124853810.json
        CIOX_RAHIX_IC20190521142219_739M8751820_124857295.json
        CIOX_RAHIX_IC20190521142219_742A6325910_124838901.json
        CIOX_RAHIX_IC20190521142219_747M8977220_124848744.json
        CIOX_RAHIX_IC20190521142219_750M5908020_124836320.json
        CIOX_RAHIX_IC20190521142219_752A6245010_124846277.json
        CIOX_RAHIX_IC20190521142219_753A7412450_124857291.json
        CIOX_RAHIX_IC20190521142219_761M8997570_124859041.json
        CIOX_RAHIX_IC20190521142219_762A6526310_124851880.json
        CIOX_RAHIX_IC20190521142219_762M8429220_124858261.json
        CIOX_RAHIX_IC20190521142219_764M8816510_124842321.json
        CIOX_RAHIX_IC20190521142219_764M9359120_124852685.json
        CIOX_RAHIX_IC20190521142219_768M8829220_124844563.json
        CIOX_RAHIX_IC20190521142219_778M5841310_124840033.json
        CIOX_RAHIX_IC20190521142219_782A7975510_124856877.json
        CIOX_RAHIX_IC20190521142219_789M8478230_124858346.json
        CIOX_RAHIX_IC20190521142219_791M8777710_124855954.json
        CIOX_RAHIX_IC20190521142219_797M5064950_124848860.json
        CIOX_RAHIX_IC20190521142219_798M8813220_124841434.json
        CIOX_RAHIX_IC20190521142219_798M8829020_124845361.json
        CIOX_RAHIX_IC20190521142219_799A7921210_124857519.json
        CIOX_RAHIX_IC20190521142219_809M9152020_124836980.json
        CIOX_RAHIX_IC20190521142219_812A6423210_124855829.json
        CIOX_RAHIX_IC20190521142219_818A5450540_124842301.json
        CIOX_RAHIX_IC20190521142219_820A7971020_124850102.json
        CIOX_RAHIX_IC20190521142219_821M9332230_124847837.json
        CIOX_RAHIX_IC20190521142219_825A7040670_124850927.json
        CIOX_RAHIX_IC20190521142219_827M8017020_124850923.json
        CIOX_RAHIX_IC20190521142219_829M6252610_124854112.json
        CIOX_RAHIX_IC20190521142219_833M9568520_124843514.json
        CIOX_RAHIX_IC20190521142219_836A6368710_124842220.json
        CIOX_RAHIX_IC20190521142219_836M9859120_124840360.json
        CIOX_RAHIX_IC20190521142219_839M8897220_124846930.json
        CIOX_RAHIX_IC20190521142219_841M7864670_124841163.json
        CIOX_RAHIX_IC20190521142219_843A7398020_124854430.json
        CIOX_RAHIX_IC20190521142219_843M6992820_124850972.json
        CIOX_RAHIX_IC20190521142219_848M6230740_124842945.json
        CIOX_RAHIX_IC20190521142219_850M9921120_124844386.json
        CIOX_RAHIX_IC20190521142219_857M7066120_124851567.json
        CIOX_RAHIX_IC20190521142219_860M8405220_124845202.json
        CIOX_RAHIX_IC20190521142219_862A7540140_124849000.json
        CIOX_RAHIX_IC20190521142219_866M5598710_124856994.json
        CIOX_RAHIX_IC20190521142219_868M7835820_124843442.json
        CIOX_RAHIX_IC20190521142219_870A6996540_124841539.json
        CIOX_RAHIX_IC20190521142219_872A5485920_124844961.json
        CIOX_RAHIX_IC20190521142219_873A6160710_124838848.json
        CIOX_RAHIX_IC20190521142219_873M9155670_124846252.json
        CIOX_RAHIX_IC20190521142219_874M8325620_124840450.json
        CIOX_RAHIX_IC20190521142219_880M7697320_124842901.json
        CIOX_RAHIX_IC20190521142219_881M5811470_124841770.json
        CIOX_RAHIX_IC20190521142219_883M5356510_124844046.json
        CIOX_RAHIX_IC20190521142219_884A7007720_124852574.json
        CIOX_RAHIX_IC20190521142219_885A2495230_124854290.json
        CIOX_RAHIX_IC20190521142219_889M5710410_124843130.json
        CIOX_RAHIX_IC20190521142219_892M8250940_124854909.json
        CIOX_RAHIX_IC20190521142219_893M7932910_124848089.json
        CIOX_RAHIX_IC20190521142219_895A6778720_124850279.json
        CIOX_RAHIX_IC20190521142219_899M5772610_124841777.json
        CIOX_RAHIX_IC20190521142219_902M7890310_124854086.json
        CIOX_RAHIX_IC20190521142219_902M9347740_124844630.json
        CIOX_RAHIX_IC20190521142219_906M7528920_124851673.json
        CIOX_RAHIX_IC20190521142219_908A5213120_124838022.json
        CIOX_RAHIX_IC20190521142219_908M9460320_124856909.json
        CIOX_RAHIX_IC20190521142219_909M6998120_124838154.json
        CIOX_RAHIX_IC20190521142219_911M6645370_124847810.json
        CIOX_RAHIX_IC20190521142219_912M9244420_124837685.json
        CIOX_RAHIX_IC20190521142219_913M6894220_124841889.json
        CIOX_RAHIX_IC20190521142219_914A7361220_124839058.json
        CIOX_RAHIX_IC20190521142219_915M5808410_124845806.json
        CIOX_RAHIX_IC20190521142219_917M8775620_124849384.json
        CIOX_RAHIX_IC20190521142219_918M9573810_124843420.json
        CIOX_RAHIX_IC20190521142219_919A5424940_124859755.json
        CIOX_RAHIX_IC20190521142219_922M9685120_124852825.json
        CIOX_RAHIX_IC20190521142219_923M6634210_124850072.json
        CIOX_RAHIX_IC20190521142219_924M5362240_124857889.json
        CIOX_RAHIX_IC20190521142219_926M5912910_124838614.json
        CIOX_RAHIX_IC20190521142219_927A7980320_124843848.json
        CIOX_RAHIX_IC20190521142219_929A6366610_124842862.json
        CIOX_RAHIX_IC20190521142219_932A7484410_124857365.json
        CIOX_RAHIX_IC20190521142219_932M6468640_124858650.json
        CIOX_RAHIX_IC20190521142219_933M6537320_124857881.json
        CIOX_RAHIX_IC20190521142219_935M5768820_124840904.json
        CIOX_RAHIX_IC20190521142219_935M7923620_124856610.json
        CIOX_RAHIX_IC20190521142219_935M8225040_124847885.json
        CIOX_RAHIX_IC20190521142219_935M8369220_124848490.json
        CIOX_RAHIX_IC20190521142219_936M8396210_124843127.json
        CIOX_RAHIX_IC20190521142219_938M5908340_124849781.json
        CIOX_RAHIX_IC20190521142219_941M5657810_124842390.json
        CIOX_RAHIX_IC20190521142219_942A7387240_124850960.json
        CIOX_RAHIX_IC20190521142219_950M5736020_124841763.json
        CIOX_RAHIX_IC20190521142219_952A7890340_124836680.json
        CIOX_RAHIX_IC20190521142219_952M5349720_124844772.json
        CIOX_RAHIX_IC20190521142219_952M7376720_124844037.json
        CIOX_RAHIX_IC20190521142219_955M7625440_124845374.json
        CIOX_RAHIX_IC20190521142219_956M7983620_124848209.json
        CIOX_RAHIX_IC20190521142219_957M5853010_124838163.json
        CIOX_RAHIX_IC20190521142219_960A7886141_124836485.json
        CIOX_RAHIX_IC20190521142219_961M9191220_124858265.json
        CIOX_RAHIX_IC20190521142219_961M9817110_124855014.json
        CIOX_RAHIX_IC20190521142219_965A6304110_124856895.json
        CIOX_RAHIX_IC20190521142219_970M9139520_124847949.json
        CIOX_RAHIX_IC20190521142219_973A7415940_124848564.json
        CIOX_RAHIX_IC20190521142219_975M8361520_124846097.json
        CIOX_RAHIX_IC20190521142219_976M9702810_124845813.json
        CIOX_RAHIX_IC20190521142219_981A5346710_124836564.json
        CIOX_RAHIX_IC20190521142219_982A8009420_124849056.json
        CIOX_RAHIX_IC20190521142219_983M5683320_124851513.json
        CIOX_RAHIX_IC20190521142219_983M5683330_124839686.json
        CIOX_RAHIX_IC20190521142219_988M8804140_124844185.json
        CIOX_RAHIX_IC20190521142219_989M9918720_124839734.json
        CIOX_RAHIX_IC20190521142219_990M5775620_124842451.json
        CIOX_RAHIX_IC20190521142219_990M8002820_124838401.json
        CIOX_RAHIX_IC20190521142219_992M8571310_124840753.json
        CIOX_RAHIX_IC20190521142219_992M9013772_124850208.json
        CIOX_RAHIX_IC20190521142219_993A5600540_124836772.json
        CIOX_RAHIX_IC20190521142219_993M8314920_124845834.json
        CIOX_RAHIX_IC20190521142219_996M5771110_124852359.json
        CIOX_RAHIX_IC20190521142219_996M5957050_124851003.json
        CIOX_RAHIX_IC20190521142219_996M6554610_124841055.json
        CIOX_RAHIX_IC20190521142219_997A5550230_124847974.json
        CIOX_RAHIX_IC20190521142219_997M9224720_124839441.json]